In [1]:
import numpy as np
import pandas as pd
import ast

In [2]:
movies = pd.read_csv('public/tmdb_5000_movies.csv')
credits = pd.read_csv('public/tmdb_5000_credits.csv') 

In [3]:
movies = movies.merge(credits,on='title')

In [4]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [5]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L 

In [6]:
movies.dropna(inplace=True)

In [7]:
movies['genres'] = movies['genres'].apply(convert)

In [8]:
movies['keywords'] = movies['keywords'].apply(convert)

In [9]:
def convert_2(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter += 1

    return L

In [10]:
movies['cast'] = movies['cast'].apply(convert_2)

In [11]:
def convert_3(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

In [12]:
movies['crew'] = movies['crew'].apply(convert_3)

In [13]:
movies['overview'] = movies['overview'].astype(str).str.split()

In [14]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ", ""))   
    return L1

In [15]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [16]:
movies_transformed = movies.drop(columns=['overview', 'genres', 'keywords', 'cast', 'crew'])

In [17]:
movies_transformed['tags'] = movies_transformed['tags'].apply(lambda x: " ".join(x))

In [18]:
import nltk

In [19]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [20]:
def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [21]:
movies_transformed['tags'] = movies_transformed['tags'].apply(stem)

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [23]:
vector = cv.fit_transform(movies_transformed['tags']).toarray()

In [24]:
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
similarity = cosine_similarity(vector)

In [26]:
def recommend(movie):
    index = movies_transformed[movies_transformed['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:2]:
        print(movies_transformed.iloc[i[0]].title)

In [27]:
recommend('Superman')

Superman II
